In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display
import ipywidgets as widgets

In [2]:
movies = pd.read_csv(r"C:\Users\ATIMANGO\Downloads\movies (2).csv")
ratings = pd.read_csv(r"C:\Users\ATIMANGO\Downloads\ratings (1).csv")

A SEARCH ENGINE

In [28]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [29]:
def clean(title):
    return re.sub("[^a-zA-Z0-9 ]","", title)

In [30]:
movies["Clean Tiltle"] = movies["title"].apply(clean)
movies.head()

,movieId,title,genres,Clean Tiltle
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995


In [31]:
vectorizer = TfidfVectorizer(ngram_range = (1,2))
tfidf = vectorizer.fit_transform(movies["Clean Tiltle"])

In [32]:
def search(title):
    title = clean(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indicies = np.argpartition(similarity, -5)[-5:]
    results = movies.iloc[indicies][::-1]
    return results

In [33]:
movie_input = widgets.Text(
    description = "Movie input",
    value = "",
    disable = "False")
movie_list = widgets.Output()
def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
        if len(title) > 2:
            display(search(title))
movie_input.observe(on_type, names="value")
display(movie_input, movie_list)

Text(value='', description='Movie input')

Output()

RECOMEMDATION SYSTEM

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [6]:
#userids who watched and liked a movieid = 1
movieid = 1
a_movieid = ratings [ (ratings["movieId"] == movieid) & (ratings["rating"] >= 4) ]["userId"].unique()

In [9]:
#userids who watched and liked a movieid = 1, other movies they like by rating >= 4
a_movieid_recs = ratings [ (ratings["userId"].isin(a_movieid)) & (ratings["rating"] >= 4) ]["movieId"]

In [11]:
a_movieid_recs = a_movieid_recs.value_counts() / len(a_movieid)

In [12]:
a_movieid_recs = a_movieid_recs[a_movieid_recs > .1]

In [17]:
#all mvoies apart from movieId = 1
all_users = ratings [ ratings["movieId"].isin(a_movieid_recs.index) & (ratings["rating"] >= 4) ]

In [22]:
all_users_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
all_users_recs

movieId
318     0.415152
296     0.381818
356     0.380303
593     0.362121
260     0.351515
          ...   
2078    0.042424
661     0.042424
1485    0.040909
348     0.037879
1391    0.037879
Name: count, Length: 276, dtype: float64

In [24]:
rec_percentages = pd.concat([a_movieid_recs, all_users_recs], axis =1)
rec_percentages.columns = ["similar", "all"]
rec_percentages["Score"] = rec_percentages["similar"] / rec_percentages["all"]
rec_percentages = rec_percentages.sort_values("Score", ascending = False)
rec_percentages

,similar,all,Score
movieId,,,
1,1.000000,0.240909,4.150943
78499,0.157233,0.048485,3.242925
3114,0.383648,0.119697,3.205159
2355,0.257862,0.081818,3.151642
1391,0.106918,0.037879,2.822642
...,...,...,...
2324,0.132075,0.110606,1.194107
79132,0.132075,0.116667,1.132075
1961,0.132075,0.118182,1.117562


In [25]:
rec_percentages.head(10).merge(movies, left_index=True, right_on = "movieId")

,similar,all,Score,movieId,title,genres
0,1.000000,0.240909,4.150943,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
7556,0.157233,0.048485,3.242925,78499,Toy Story 3 (2010),Adventure|Animation|Children|Comedy|Fantasy|IMAX
2506,0.383648,0.119697,3.205159,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy
1866,0.257862,0.081818,3.151642,2355,"Bug's Life, A (1998)",Adventure|Animation|Children|Comedy
1129,0.106918,0.037879,2.822642,1391,Mars Attacks! (1996),Action|Comedy|Sci-Fi
313,0.106918,0.037879,2.822642,348,Bullets Over Broadway (1994),Comedy
1020,0.144654,0.054545,2.651992,1266,Unforgiven (1992),Drama|Western
419,0.125786,0.048485,2.594340,471,"Hudsucker Proxy, The (1994)",Comedy
567,0.106918,0.042424,2.520216,661,James and the Giant Peach (1996),Adventure|Animation|Children|Fantasy|Musical
4610,0.295597,0.119697,2.469549,6377,Finding Nemo (2003),Adventure|Animation|Children|Comedy


In [35]:
def find_similar_movies(movie_id):
    a_movieid = ratings[ (ratings["movieId"] == movie_id) & (ratings["rating"] > 4) ]["userId"].unique()
    a_movieid_recs = ratings[(ratings["userId"].isin(a_movieid)) & (ratings["rating"] > 4)]['movieId']
    a_movieid_recs = a_movieid_recs.value_counts() / len (a_movieid)
    a_movieid_recs = a_movieid_recs[a_movieid_recs > .1]
    all_users = ratings[ (ratings["movieId"].isin(a_movieid_recs.index)) & (ratings["rating"] > 4) ]
    all_users_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    rec_percentages = pd.concat([a_movieid_recs, all_users_recs], axis =1)
    rec_percentages.columns = ["similar", "all"]
    rec_percentages["Score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("Score", ascending = False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on = "movieId")[["Score", "title", "genres"]]

In [37]:
movie_input_name = widgets.Text(
    description = "Movie Title",
    value = "",
    disable = False
)
recommendation_list = widgets.Output()
def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len (title) > 2:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))
movie_input_name.observe(on_type, names="value")
display(movie_input_name, recommendation_list) 

Text(value='', description='Movie Title')

Output()